<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/House_Price_ML_CCG/Dummies_House_Price_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:

from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("all_dumimies_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

In [ ]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

In [5]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Assign the data to X and y

X = df[["1stFlrSF", 
        "GrLivArea", 
        "LotArea", 
        "GarageArea", 
        "BsmtUnfSF", 
        "TotalBsmtSF", 
        "LotFrontage", 
        "GarageYrBlt", 
        "MoSold", 
        "YearBuilt"]]
y = df["SalePrice"]
print(X.shape, y.shape)

In [12]:
# Create the model using RandomForestRegressor

### BEGIN SOLUTION
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=20)
### END SOLUTION

In [ ]:
# Finalize the model by fitting to the entire training dataset and calculate the scores for the training data

### BEGIN SOLUTION
model.fit(X, y)
score = model.score(X, y)

### END SOLUTION 

print(f"Score: {score}")


In [ ]:
# Load in predict.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/{NEW CSV}}"
spark.sparkContext.addFile(url)

dfp = spark.read.option('header', 'true').csv(SparkFiles.get("{NEW CSV}"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
dfp.show(10)

#Convert to Pandas to allow train_test_split usage
dfp = df.select("*").toPandas()
dfp = df.dropna(how="any")

In [ ]:
X_predict = dfp[["1stFlrSF", 
        "GrLivArea", 
        "LotArea", 
        "GarageArea", 
        "BsmtUnfSF", 
        "TotalBsmtSF", 
        "LotFrontage", 
        "GarageYrBlt", 
        "MoSold", 
        "YearBuilt"]]

y_predict = model.predict(X_predict)

for i in X_predict:
	print("Predicted=%s" % (y_predict[i]))